In [1]:
//////////////////////////////////////////////////////
// Classification sur SpamDataBase
///////////////////////////////////////////////////
import sys.process._
import org.apache.spark.sql.types._
import org.apache.spark.ml.linalg._
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.DoubleType
import collection.mutable
import org.apache.spark.ml.clustering.KMeans

// Fonction qui produit le schéma du Dataframe sans la dernière colonne
def genSpamFields(from: Int, to: Int) = for (i <- from until to)
                         yield StructField("val"+i.toString, DoubleType, true)
// Génération des StructField successifs correspondant aux 57 premières colonnes
val spamFields = genSpamFields(0, 57)
// Construction du schéma complet du Dataframe (incluant la dernière colonne)
val spamSchema = StructType(spamFields).add("label", DoubleType, true)
// Lecture des données et création du Dataframe
val spamDF = spark.read.format("csv").schema(spamSchema).load("data/spambase.data").cache()

val n: Int = 0
val m:Int = 56

val featureCols = spamDF.columns.slice(n, m)

val exprs = featureCols.map(c => col(c).cast("double"))
val assembler = new VectorAssembler()
  .setInputCols(featureCols)
  .setOutputCol("features")

val spamVector = assembler.transform(spamDF.select(exprs: _*)).select($"features")
val kmeans = new KMeans().setK(5).setMaxIter(200).setSeed(1L)
val modele = kmeans.fit(spamVector)

val indices = modele.transform(spamVector)
spamVector.select("features").map(p => p.toString.filter(c => c != '[' & c != ']'& c != ')'& c != '(')).write.text("repvecteur")
Process(Seq("bash","-c","cat repvecteur/part* > spamvecteur")).!

indices.select("prediction").map(p => p.toString.filter(c => c != '[' & c != ']'& c != ')'& c != '(')).write.text("repindice")
Process(Seq("bash","-c","cat repindice/part* > spamgroupes")).!
Process(Seq("bash","-c","rm -rf repvecteur;rm -rf repindice")).!
Process(Seq("bash","-c","paste --delimiters=',' spamvecteur spamgroupes > spam.Gnuplot")).!


spamFields = Vector(StructField(val0,DoubleType,true), StructField(val1,DoubleType,true), StructField(val2,DoubleType,true), StructField(val3,DoubleType,true), StructField(val4,DoubleType,true), StructField(val5,DoubleType,true), StructField(val6,DoubleType,true), StructFiel...


genSpamFields: (from: Int, to: Int)scala.collection.immutable.IndexedSeq[org.apache.spark.sql.types.StructField]


Vector(StructField(val0,DoubleType,true), StructField(val1,DoubleType,true), StructField(val2,DoubleType,true), StructField(val3,DoubleType,true), StructField(val4,DoubleType,true), StructField(val5,DoubleType,true), StructField(val6,DoubleType,true), StructField(val7,DoubleType,true), StructField(val8,DoubleType,true), StructField(val9,DoubleType,true), StructField(val10,DoubleType,true), StructField(val11,DoubleType,true), StructField(val12,DoubleType,true), StructField(val13,DoubleType,true), StructField(val14,DoubleType,true), StructField(val15,DoubleType,true), StructField(val16,DoubleType,true), StructField(val17,DoubleType,true), StructField(val18,DoubleType,true), StructField(val19,DoubleType,true), StructField(val20,DoubleType,true), StructField(val21,DoubleType,true), StructField(val22,DoubleType,true), StructField(val23,DoubleType,true), StructField(val24,DoubleType,true), StructField(val25,DoubleType,true), StructField(val26,DoubleType,true), StructField(val27,DoubleType,tr

In [ ]:
//Pipeline pour Kmeans
import sys.process._
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.StandardScaler
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler

// Construction de DataFrame avec les données de Spambase
def genSpamFields(from: Int, to: Int) = for (i <- from until to)
                         yield StructField("val"+i.toString, DoubleType, true)
val spamFields = genSpamFields(0, 57)
val spamSchema = StructType(spamFields).add("label", DoubleType, true)
val spamDF = spark.read.format("csv").schema(spamSchema).load("data/spambase.data")
val colsEntree = {for (i <- 0 until 57) yield "val"+i.toString}.toArray
val assembleur = new VectorAssembler().setInputCols(colsEntree).setOutputCol("features")
val spamDFA = assembleur.transform(spamDF).cache()
// Partitionnement en données de train (80%) et données de test (20%)
val spamSplits = spamDFA.randomSplit(Array(0.8, 0.2))

// Construction d'un pipeline
val scaler = new StandardScaler().setInputCol("features")
                                        .setOutputCol("scaledFeatures")
                                        .setWithStd(true)
                                        .setWithMean(true)
val kmeansNS = new KMeans().setFeaturesCol(scaler.getOutputCol)
                                  .setPredictionCol("predictionNS")
                                  .setK(5)
                                  .setMaxIter(200)
                                  .setSeed(1L)
val pipeline = new Pipeline().setStages(Array(scaler, kmeansNS))

// Estimation des modèles du pipeline (sur données de train):
//   le premier modèle (normalisation) est estimé en premier,
//   ensuite il est utilisé comme transformer pour modifier les données
//   et c'est le second modèle qui est estimé (clustering)
//
val modeleKMNS = pipeline.fit(spamSplits(0))

// Application du modèle pour prédire les groupes des données de test
//
val indicesSpamTest = modeleKMNS.transform(spamSplits(1))
                                       .select("scaledFeatures","predictionNS")

indicesSpamTest.select("scaledFeatures")
                    .map(v => v.toString.filter(c => c != '[' & c != ']'))
                    .write.text("data/vecteursTestSpam")
indicesSpamTest.select("predictionNS")
                      .map(p => p.toString.filter(c => c != '[' & c != ']'))
                      .write.text("data/indicesTestSpam")
Process(Seq("bash","-c","cat data/vecteursTestSpam/part* > data/vecteurs")).!
Process(Seq("bash","-c","cat data/indicesTestSpam/part* > data/idgroupes")).!
Process(Seq("bash","-c","rm -rf data/vecteursTestSpam;rm -rf data/indicesTestSpam")).!
Process(Seq("bash","-c","paste --delimiters=',' data/vecteurs data/idgroupes > spam_Gnuplot.txt")).!



// Un pipeline peut être sauvegardé (avant estimation)
pipeline.write.overwrite().save("spark-pipeline-normEtKMeans")

// Un modèle issu d'un pipeline (après estimation) peut être sauvegardé
modeleKMNS.write.overwrite().save("spark-modele-clustering-spam")

// Un modèle sauvegardé peut être chargé pour être employé
val memeModele = PipelineModel.load("spark-modele-clustering-spam")

In [28]:
spamSplits(1)

[val0: double, val1: double ... 57 more fields]

In [31]:
// on fait une ACP pour avoir un K-means sur les composantes => reduction à 3 dimensions significatives
import org.apache.spark.ml.feature.PCA

// Construction et application d'une nouvelle instance d'estimateur PCA
val pca = new PCA().setInputCol("Features").setOutputCol("pcaFeatures")
                          .setK(3).fit(spamSplits(1).select("Features"))

// Application du « transformateur » PCAModel résultant de l'estimation
val resultat = pca.transform(spamSplits(1).select("Features")).select("pcaFeatures")

// Les 3 plus grandes valeurs propres (exprimées en proportion de variance expliquée)
pca.explainedVariance


pca = pca_db1df9d91d11
resultat = [pcaFeatures: vector]


[0.9667801236430948,0.032715960787922375]

In [32]:
resultat.select("pcaFeatures").map(p => p.toString.filter(c => c != '[' & c != ']'))
                      .write.text("data/PCASpam")
Process(Seq("bash","-c","cat data/PCASpam/part* > data/vecteursPCA")).!
Process(Seq("bash","-c","rm -rf data/PCASpam")).!
Process(Seq("bash","-c","paste --delimiters=',' data/vecteursPCA data/idgroupes > spam_PCA_Gnuplot.txt")).!

0